In [29]:
import json
import glob
import csv
import string

In [30]:
filepath = 'comics2event.csv'

In [31]:
out_put = open(filepath, "w+", newline='')
datafile = csv.writer(out_put)      
header = ['event_id','comic_id']
datafile.writerow(header)

19

In [32]:
files =  glob.glob('./comic_json/*.json')

In [33]:
with open(files[0]) as data_file:
    data = json.load(data_file)['data']['results']
files[0] 

'./comic_json/comic29600.json'

In [34]:
data[0]['events']['available'] > data[0]['events']['returned'] 

False

In [35]:
data[20]['id']

15285

In [36]:
for i in data[20]['events']['items']:
    print(i['resourceURI'].split('/')[-1])

In [37]:
i = 0
api = []
not_api = []
for fn in files:
    with open(fn) as data_file:
        data = json.load(data_file)['data']['results']
        for comic in data:
            idd = comic['id']
            avail = comic['events']['available']
            returned = data[0]['events']['returned'] 
            if avail > 20:
                api.append(idd)
                continue
            
            not_api.append(idd)
            if(len(comic['events']['items']) == 0):
                i = i + 1
                
            for char in comic['events']['items']:
                row = []
               
                row.append(char['resourceURI'].split('/')[-1])
                row.append(idd)
                datafile.writerow(row)
            
                
        
        


In [38]:
api

[]

In [39]:
len(api)

0

In [40]:
len(not_api)

36759

In [47]:
i

4319

In [48]:
out_put.close()

In [49]:
import requests
import hashlib
import datetime
import string
import csv

In [50]:
public_key = "2990a1aae96d40df9612767f23e2cf01"
private_key = "3952fc6b62f02a30c86ee6ced917550f08f743ec"

In [51]:
def get_TS_HASH( pub_key, priv_key):
    ts = str(datetime.datetime.now()).replace(" ", "").replace(".", "").replace("-", "").replace(":", "")
    m = hashlib.md5()
    h = ts + priv_key + pub_key
    m.update( h.encode('utf-8'))
    h = m.hexdigest()
    return ts, h
    

In [52]:
def make_url(base_url,limit,offset, pub_key, priv_key):
    ts, h = get_TS_HASH( pub_key, priv_key)
    get_url = base_url + '?ts=' + ts + '&limit=' + str(limit) + '&offset=' + str(offset) + "&apikey=" + pub_key + '&hash=' + h
    return get_url

In [53]:
out_put = open(filepath, "a", newline='')
datafile = csv.writer(out_put) 

In [54]:
comic_id = api[:]

In [55]:
count = 0

In [56]:
comic_charID_url = 'http://gateway.marvel.com:80/v1/public/comics/%s/creators'

In [57]:
r = requests.get(make_url(comic_charID_url % str(comic_id[1]), 100, count, public_key, private_key))

In [58]:
data = r.json()['data']
total = data['total']

In [59]:
#idd = comic_id.index(5589)

In [60]:
#idd = idd + 2

In [61]:
#comic_id = comic_id[idd:len(comic_id)]

In [62]:
for i in range(len(comic_id)):
    count = 0
    url = (comic_charID_url % comic_id[i])
    r = requests.get(make_url(url, 100, count, public_key, private_key))
    if(r.status_code != 200):
        print("something wrong!!! error code of %d"% r.status_code)
        break
    data = r.json()['data']
    total = data['total']
    while(1):
        count = count + data['count']
        for char in data['results']:
            row = []
            row.append(comic_id[i])
            row.append(char['id'])
            datafile.writerow(row)
        if(count > total):
            break
        if(r.status_code != 200):
            print("something wrong!!! error code of %d"% r.status_code)
            break
        data = r.json()['data']
        
        
        
        
        
    
    
    

In [63]:
out_put.close()